## Analyze whether SNWD varies more from year to year or from place to place.

In [8]:
import pandas as pd
import numpy as np
import urllib
import math

In [43]:
import findspark
findspark.init()

from pyspark import SparkContext
sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [44]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [45]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSBSBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
362
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+-------------------+------------------+------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|              res_2|             res_3|          res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+-------------------+------------------+------------------+-----------+-----------+------+--------------------+------+
|1389.2409573309023| 67.23764973963051|-273.6827183050143|   2801.1|SSSSBSBB| 32.7904|-105.8204|       SNWD|0.2805995371466946|0.27678531228276415| 0.254408619259497|0.9622207218664723|US1NMO

In [46]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [57]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1 FROM weather


station    year      coeff_1
0  US1NMOT0039  2007.0  1389.240957
1  USC00291927  1983.0  2457.901690
2  USC00291931  2010.0  4603.571127
3  US1NMOT0056  2010.0  6948.582906
4  USC00291931  1998.0  2712.868387

In [58]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station  US1NMDA0112  US1NMOT0039  US1NMOT0056  USC00022754  USC00290199  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1912.0           NaN          NaN          NaN          NaN          NaN   
1913.0           NaN          NaN          NaN          NaN          NaN   
1915.0           NaN          NaN          NaN          NaN          NaN   
1916.0           NaN          NaN          NaN          NaN          NaN   
1917.0           NaN          NaN          NaN          NaN          NaN   
1918.0           NaN          NaN          NaN          NaN          NaN   
1923.0           NaN          NaN          NaN          NaN          NaN   

station  USC00290268  USC00291252  USC00291286  USC00291790  USC00291910  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1912.0           NaN          NaN          NaN          NaN          NaN   
1913.0           NaN          NaN          NaN          NaN          NaN   
1915.0           NaN          NaN          NaN          NaN          NaN   
1916.0           NaN          NaN          NaN          NaN          NaN   
1917.0           NaN          NaN          NaN          NaN          NaN   
1918.0           NaN          NaN          NaN          NaN          NaN   
1923.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00299686  USC00299691  USC00299697  USC00299720  \
year        ...                                                            
1901.0      ...               NaN          NaN          NaN          NaN   
1905.0      ...               NaN          NaN          NaN          NaN   
1909.0      ...               NaN          NaN          NaN          NaN   
1912.0      ...               NaN          NaN          NaN          NaN   
1913.0      ...               NaN          NaN          NaN          NaN   
1915.0      ...               NaN          NaN   363.968307          NaN   
1916.0      ...               NaN          NaN    97.101051          NaN   
1917.0      ...               NaN          NaN          NaN          NaN   
1918.0      ...               NaN          NaN          NaN          NaN   
1923.0      ...               NaN          NaN          NaN          NaN   

station  USC00299806  USW00023002  USW00023039  USW00023046  USW00093045  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1912.0           NaN          NaN          NaN          NaN          NaN   
1913.0           NaN          NaN          NaN          NaN          NaN   
1915.0           NaN          NaN          NaN          NaN          NaN   
1916.0           NaN          NaN          NaN          NaN          NaN   
1917.0           NaN          NaN          NaN          NaN          NaN   
1918.0           NaN          NaN          NaN          NaN          NaN   
1923.0           NaN          NaN          NaN          NaN          NaN   

station  USW00093063  
year                  
1901.0           NaN  
1905.0           NaN  
1909.0           NaN  
1912.0           NaN  
1913.0           NaN  
1915.0           NaN  
1916.0           NaN  
1917.0           NaN  
1918.0           NaN  
1923.0           NaN  

[10 rows x 55 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [59]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  557.468013648
RMS removing mean-by-station=  396.027732492
RMS removing mean-by-year   =  455.207244626


In [50]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 167.950634625
0 after removing mean by year    = 136.586038427
0 after removing mean by stations= 132.796386253
1 after removing mean by year    = 132.106135683
1 after removing mean by stations= 131.896684003
2 after removing mean by year    = 131.809490304
2 after removing mean by stations= 131.763833849
3 after removing mean by year    = 131.736541191
3 after removing mean by stations= 131.719111817
4 after removing mean by year    = 131.707616332
4 after removing mean by stations= 131.699905909
